进行同义词替换：
1. 使用nltk语义库替换valid.json
2. 使用gpt替换valid.json
3. 使用detect model替换test.json，再比较与原test.json文本的差异，使用均方差来评估clean和dirty最终使用auc评分

## 1. 使用nltk

In [ ]:
import json
import nltk
from nltk.corpus import wordnet

# 下载所需的NLTK数据
nltk.download('wordnet')
nltk.download('omw-1.4')

# 同义词替换函数
def synonym_replacement(text):
    words = nltk.word_tokenize(text)
    new_words = []
    for word in words:
        synonyms = wordnet.synsets(word)
        if synonyms:
            synonym = synonyms[0].lemmas()[0].name()
            new_words.append(synonym)
        else:
            new_words.append(word)
    return ' '.join(new_words)

# 加载原始的验证集
input_file_path = 'dataset/valid.json'
output_file_path = 'dataset/synonym_replacement_valid.json'

with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 对每个文本进行同义词替换
for entry in data:
    entry['synonym_replacement'] = synonym_replacement(entry['text'])

# 保存到新的json文件
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

# 提示操作完成
print("同义词替换已完成并保存到新的json文件中。")


## 2.使用gpt

In [ ]:
import json
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# 加载GPT-2模型和tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

# 同义词替换函数
def synonym_replacement_gpt(text, num_return_sequences=1):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    
    # 打印调试信息
    print(f"Input text: {text}")
    print(f"Input IDs: {input_ids}")
    print(f"Input length: {len(input_ids[0])}")
    
    input_length = len(input_ids[0])
    max_new_tokens = 50

    if input_length >= 1024:
        # 如果输入长度已经超过或达到最大长度，则截断输入
        input_ids = input_ids[:, :1024 - max_new_tokens]
        input_length = len(input_ids[0])
    
    # 确保生成的总长度不会超过 1024
    max_new_tokens = min(max_new_tokens, 1024 - input_length)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids, 
            max_new_tokens=max_new_tokens,
            num_return_sequences=num_return_sequences, 
            num_beams=5, 
            no_repeat_ngram_size=2, 
            early_stopping=True
        )
    
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return generated_texts[0]

# 加载验证集数据
input_file_path = 'dataset/valid.json'
output_file_path = 'dataset/synonym_replacement_valid_gpt.json'

with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 对每个文本进行同义词替换
for entry in data:
    entry['text'] = synonym_replacement_gpt(entry['text'])

# 保存到新的json文件
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

# 提示操作完成
print(f"同义词替换已完成并保存到新的json文件中：{output_file_path}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input text: A water company has blamed more people working from home post-pandemic for a new hosepipe ban.South East Water, which supplies more than 2m homes and businesses, will impose the first hosepipe ban of the summer on Monday, affecting households across Kent and Sussex.The company’s chief executive, David Hinton, said that people working from home was a “key factor” behind the ban, as it has “increased drinking water demand”.In a letter to customers, he wrote: “Over the past three years the way in which drinking water is being used across the south-east has changed considerably.“The rise of working from home has increased drinking water demand in commuter towns by around 20% over a very short period, testing our existing infrastructure.”Hinton also blamed low rainfall since April as well as a recent spell of hot weather which he said led to a spike in demand for drinking water.“Our reservoir and aquifer stocks of raw water, essential to our water supply but not ready to be used

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input text: The mighty aurochs have gone, as have the tarpan horses and the wild boars, but modern-day substitutes have been drafted in to recreate a large open “savannah” on heathland in Dorset.Instead of aurochs, considered the wild ancestor of domestic cattle, 200 red Devon cattle are to be found roaming the Purbeck Heaths, while Exmoor ponies are stand-ins for the tarpan horses and curly coated Mangalitsa pigs are doing the sort of rooting around that boars used to excel at here.The idea of the project is to create more of the sort of habitat where precious species such as the sand lizard, southern damselfly and heath tiger beetle can thrive.Two Exmoor ponies at Purbeck Heaths. Photograph: National Trust ImagesIt comes three years after the UK’s first “super national nature reserve” was created in Dorset, knitting together 3,400 hectares (8,400 acres) of priority habitat. Within the super reserve, 1,370 hectares of open “savannah” for free-ranging, grazing animals as it would have 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Input text: A crime scene has been established after a fire engulfed a plastics factory in Melbourne’s south-east, sending a plume of toxic smoke into surrounding suburbs.A police officer raised the alarm when he noticed smoke coming from the Olive Grove building while patrolling at 5.10am on Wednesday morning.Buddhist temple in south-east Melbourne gutted by fireRead moreFire crews extinguished the blaze but not before “significant damage” was caused to the unoccupied single-storey premises, according to Victoria police.A crime scene has been established and motorists were being asked to avoid the area.Police said the cause of the fire was yet to be determined and fire investigators would attend the scene.JUST IN: Vision of the factory fire this morning before firefighters arrived. It was a huge blaze which ripped through the plastic factory in Keysborough incredibly quickly. Advice warning in place for residents nearby, traffic still moving on the Dandenong bypass @sunriseon7 pic.twi

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input text: The UK’s fruit trees are under threat as a result of the climate crisis because plant diseases that thrive in warm weather are becoming more common.Each year, the Royal Horticultural Society (RHS) compiles a list of the most common plant diseases identified by its almost half a million members. Gardeners take pictures or samples of afflicted trees, crops or flowers and send them in to the plant pathologists, who can identify the disease.This year, seven of 2022’s 10 most prevalent garden diseases relate to fruit, which the RHS says is the highest it has seen. Apple and pear canker also entered the top 10 for the first time in recent years.Traditional British garden under threat from extreme heat, says RHSRead moreDr Liz Beal, a plant pathologist at the RHS, said: “This is a direct result of last year’s extreme summer heat, which caused many plants to become stressed and therefore more susceptible to problems when rain, coupled with continuing mild temperatures in the autumn

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input text: More than 50 billboards and bus stop adverts drawing attention to the Liverpool FC sponsor Standard Chartered’s links to the fossil fuel industry have appeared across the city, as a number of activist groups targeted the bank ahead of its annual general meeting on Wednesday.One poster design installed outside Anfield, featuring the Liverpool manager Jürgen Klopp and player Mohamed Salah, reads: “Give Standard Chartered the red card.”Meanwhile another campaign group put up a fake Standard Chartered website – described as “extremely convincing” by those who saw it – which “announced” that the bank would “end all support for coal in 2021 and all fossil fuel infrastructure by 2023”. The hoax had been designed by Fridays For Future and the Yes Men. At a morning press conference the real group behind the claim revealed themselves.The campaign group Market Forces, which focuses on finance for fossil fuels, also said it would be using the shares it owns in the bank to lodge a share

## 3.

In [5]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, GPT2Tokenizer, GPT2LMHeadModel
from sklearn.metrics import roc_auc_score
from torch.nn.functional import softmax

# 加载预训练的BERT模型
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
bert_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')
bert_model.eval()  # 评估模式

# 加载预训练的GPT-2模型用于同义词替换
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')
gpt_model.eval()

# 同义词替换函数
def synonym_replacement_gpt(text, num_return_sequences=1):
    input_ids = gpt_tokenizer.encode(text, return_tensors='pt')
    with torch.no_grad():
        outputs = gpt_model.generate(
            input_ids, 
            max_length=len(input_ids[0]) + 50, 
            num_return_sequences=num_return_sequences, 
            num_beams=5, 
            no_repeat_ngram_size=2, 
            early_stopping=True
        )
    generated_texts = [gpt_tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return generated_texts[0]

# 获取模型输出
def get_model_output(text):
    inputs = bert_tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return softmax(outputs.logits, dim=1)

# 对比原始文本和同义改写文本
def compare_texts(original_text, rewritten_text):
    original_output = get_model_output(original_text)
    rewritten_output = get_model_output(rewritten_text)
    return torch.nn.functional.mse_loss(original_output, rewritten_output).item()

# 加载数据
input_file_path = 'dataset/valid.json'  # 替换为你的数据文件路径
output_file_path = 'dataset/synonym_detection_results.json'

with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 处理数据
for entry in data:
    entry['synonym_replacement'] = synonym_replacement_gpt(entry['text'])
    entry['difference'] = compare_texts(entry['text'], entry['synonym_replacement'])

# 假设阈值可以通过统计分析或其他方法确定
threshold = 0.1
for entry in data:
    entry['predicted_label'] = 1 if entry['difference'] > threshold else 0

# 计算AUC作为评价指标
true_labels = [1 if entry['label'] == 'dirty' else 0 for entry in data]
predicted_labels = [entry['predicted_label'] for entry in data]
auc_score = roc_auc_score(true_labels, predicted_labels)
print(f"AUC Score: {auc_score}")

# 保存结果
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)


D:\Anaconda\envs\llmhw\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like bert-base-uncased is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.